<a href="https://colab.research.google.com/github/ShifaliSanthosh/data_engineer_bajaj/blob/main/DATA_ENGINEER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json

with open('DataEngineeringQ2.json', 'r') as file:
    data = json.load(file)

rows = []
for record in data:
    base_data = {
        "appointmentId": record.get("appointmentId"),
        "patientId": record["patientDetails"].get("_id"),
        "firstName": record["patientDetails"].get("firstName"),
        "lastName": record["patientDetails"].get("lastName"),
        "gender": record["patientDetails"].get("gender"),
        "birthDate": record["patientDetails"].get("birthDate"),
        "phoneNumber": record.get("phoneNumber"),
    }

    for medicine in record["consultationData"]["medicines"]:
        row = base_data.copy()
        row.update({
            "medicineId": medicine.get("medicineId"),
            "medicineName": medicine.get("medicineName"),
            "frequency": medicine.get("frequency"),
            "duration": medicine.get("duration"),
            "durationIn": medicine.get("durationIn"),
            "instruction": medicine.get("instruction"),
            "isActive": medicine.get("isActive"),
        })
        rows.append(row)
    #process chief complaints
    if not record["consultationData"]["medicines"]:
        for complaint in record["consultationData"]["chiefComplaints"]:
            row = base_data.copy()
            row.update({
                "chiefComplaint": complaint.get("symptomSnomedName"),
            })
            rows.append(row)

df = pd.DataFrame(rows)
nf = df.drop_duplicates()

csv_path = "final_output.csv"
nf.to_csv(csv_path, index=False)

print(f"CSV file created at {csv_path}")

CSV file created at final_output.csv


In [ ]:
import pandas as pd

df = pd.read_csv('final_output.csv')


missing_percentage = df[['firstName', 'lastName', 'birthDate']].isnull().mean() * 100
print("Missing values percentage:\n", missing_percentage)

mode_gender = df['gender'].mode()[0]
df['gender'] = df['gender'].fillna(mode_gender)


female_percentage = (df['gender'].value_counts(normalize=True).get('F', 0)) * 100
print(f"\nPercentage of Female after imputation: {female_percentage:.2f}%")


df['birthDate'] = pd.to_datetime(df['birthDate'], errors='coerce')


print("Rows with invalid birthDate:")
print(df[df['birthDate'].isnull()].head())


df['birthDate'] = df['birthDate'].dt.tz_localize(None)


df['age'] = (pd.to_datetime('today') - df['birthDate']).dt.days // 365

print("Sample of calculated ages:")
print(df[['birthDate', 'age']].head())


def categorize_age(age):
    if age <= 12:
        return 'Child'
    elif age <= 19:
        return 'Teen'
    elif age <= 59:
        return 'Adult'
    elif age >= 60:
        return 'Senior'
    else:
        return None

df['ageGroup'] = df['age'].apply(categorize_age)

print("Unique age groups:")
print(df['ageGroup'].value_counts())

adult_count = df[df['ageGroup'] == 'Adult'].shape[0]
print(f"\nCount of Adults: {adult_count}")

avg_medicines = df.groupby('appointmentId')['medicineName'].count().mean()
print(f"\nAverage number of medicines prescribed per patient: {avg_medicines:.2f}")


medicine_counts = df['medicineName'].value_counts()
third_most_frequent_medicine = medicine_counts.index[2]
print(f"\n3rd most frequently prescribed medicine: {third_most_frequent_medicine}")


active_inactive_counts = df['isActive'].value_counts(normalize=True) * 100
print("\nPercentage distribution of active vs inactive medicines:\n", active_inactive_counts)

Missing values percentage:
 firstName     0.000000
lastName     69.565217
birthDate    31.884058
dtype: float64

Percentage of Female after imputation: 66.67%
Rows with invalid birthDate:
  appointmentId  patientId firstName lastName gender birthDate   phoneNumber  \
0     40d2-9c9f  T6hb630b3       Css      NaN      F       NaT  9.668690e+10   
1     40d2-9c9f  T6hb630b3       Css      NaN      F       NaT  9.668690e+10   
4      g3Wt5c9f   TjhB4373     Shila      Das      F       NaT  7.787205e+09   
5      g3Wt5c9f   TjhB4373     Shila      Das      F       NaT  7.787205e+09   
6      g3Wt5c9f   TjhB4373     Shila      Das      F       NaT  7.787205e+09   

   medicineId medicineName frequency  duration durationIn instruction  \
0    619404.0            A     1-0-1      90.0       DAYS  AFTER MEAL   
1    619804.0            B     1-0-1      90.0       DAYS  AFTER MEAL   
4    619404.0            B     1-0-1      10.0       DAYS  AFTER MEAL   
5    619404.0            A     1-0-1   

In [ ]:
import pandas as pd

df = pd.read_csv('final_output.csv')


def is_valid_mobile(phone):
    if pd.isna(phone):
        return False

    phone = str(phone)

    if phone.startswith('+91'):
        phone = phone[3:]
    elif phone.startswith('91'):
        phone = phone[2:]

    if len(phone) == 10 and phone.isdigit():
        num = int(phone)
        if 6000000000 <= num <= 9999999999:
            return True
    return False

df['isValidMobile'] = df['phoneNumber'].apply(is_valid_mobile)

valid_count = df['isValidMobile'].sum()
print(f"Number of valid phone numbers: {valid_count}")

df.to_csv('final_output_with_valid_mobiles.csv', index=False)

from google.colab import files
files.download('final_output_with_valid_mobiles.csv')


Number of valid phone numbers: 0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

df = pd.read_csv('final_output_with_valid_mobiles.csv')

df['birthDate'] = pd.to_datetime(df['birthDate'], errors='coerce')
df['birthDate'] = df['birthDate'].dt.tz_localize(None)


df['age'] = (pd.to_datetime('today').tz_localize(None) - df['birthDate']).dt.days // 365

med_count = df.groupby('appointmentId')['medicineName'].count().reset_index()
med_count.rename(columns={'medicineName': 'numMedicines'}, inplace=True)

df_age = df[['appointmentId', 'age']].drop_duplicates()
merged_data = pd.merge(med_count, df_age, on='appointmentId')

correlation = merged_data['numMedicines'].corr(merged_data['age'], method='pearson')

correlation = round(correlation, 2)

print(correlation)


-0.27
